In [2]:
import numpy as np
import pandas as pd
import re
import csv

In [3]:
url_regex = "^(https?|ftp):\/\/([^\/]+)\/(([^\/?]+\/)*)?([^\/?]+\.\w+)?(\?([^=]+=[^&]+)(&([^=]+=[^&]+))*)?$"
url = "https://test.co.uk/?help=1"
match = re.match(url_regex, url)

def extract_urls_from_email(email_content):
    # Regex to match URLs
    url_pattern = r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"

    # Find all URLs in the email content
    urls = re.findall(url_pattern, email_content)
    return urls

def getNumSuspiciousKeywords(email_content):
    # need to somehow account for bypass word filter by using different letter
    text_lower = email_content.lower()
    keywords = ["urgent update", "pay now"]
    keyword_count = 0
    
    # Iterate through the keywords and count occurrences in the text
    for keyword in keywords:
        # Check if the keyword (case-insensitive) is present in the text
        if keyword.lower() in text_lower:
            keyword_count += 1
    
    return keyword_count

def getNumShortedUrls(email_content):
    short_domains = ['goo.gl', 'jmp.by']
    pattern = r'https?://(?:{})/\S+'.format('|'.join(re.escape(domain) for domain in short_domains))
    urls = re.findall(pattern, email_content)
    
    # Count the number of URLs found
    return len(urls)

def getNumIpAddresses(email_content):
    ipv4_regex = "\b(?:\d{1,3}\.){3}\d{1,3}\b"
    ipv6_regex = "(([0-9a-fA-F]{1,4}:){7,7}[0-9a-fA-F]{1,4}|([0-9a-fA-F]{1,4}:){1,7}:|([0-9a-fA-F]{1,4}:){1,6}:[0-9a-fA-F]{1,4}|([0-9a-fA-F]{1,4}:){1,5}(:[0-9a-fA-F]{1,4}){1,2}|([0-9a-fA-F]{1,4}:){1,4}(:[0-9a-fA-F]{1,4}){1,3}|([0-9a-fA-F]{1,4}:){1,3}(:[0-9a-fA-F]{1,4}){1,4}|([0-9a-fA-F]{1,4}:){1,2}(:[0-9a-fA-F]{1,4}){1,5}|[0-9a-fA-F]{1,4}:((:[0-9a-fA-F]{1,4}){1,6})|:((:[0-9a-fA-F]{1,4}){1,7}|:)|fe80:(:[0-9a-fA-F]{0,4}){0,4}%[0-9a-zA-Z]{1,}|::(ffff(:0{1,4}){0,1}:){0,1}((25[0-5]|(2[0-4]|1{0,1}[0-9]){0,1}[0-9])\.){3,3}(25[0-5]|(2[0-4]|1{0,1}[0-9]){0,1}[0-9])|([0-9a-fA-F]{1,4}:){1,4}:((25[0-5]|(2[0-4]|1{0,1}[0-9]){0,1}[0-9])\.){3,3}(25[0-5]|(2[0-4]|1{0,1}[0-9]){0,1}[0-9]))"
    
    ipv4_addresses = re.findall(ipv4_regex, email_content)
    ipv6_addresses = re.findall(ipv6_regex, email_content)
    
    return len(ipv4_addresses) + len(ipv6_addresses)

if match:
    protocol = match.group(1)
    domain = match.group(2)
    directory = match.group(3) if match.group(3) else ""  # Handle optional directory
    file = match.group(4) if match.group(4) else ""
    parameters = match.group(6) if match.group(6) else ""  # Handle optional parameters

    print("Protocol:", protocol)
    print("Domain:", domain)
    print("Directory:", directory)
    print("File:", file)
    print("Parameters:", parameters)



Protocol: https
Domain: test.co.uk
Directory: 
File: 
Parameters: ?help=1


In [8]:
csv.field_size_limit(100000000)
def read_csv_into_objects(file_path):
    rows = []
    with open(file_path, 'r') as csvfile:
        csv_reader = csv.reader(csvfile)
        header = next(csv_reader)  # Skip header
        print(header)
        for row in csv_reader:
            rows.append(row)
    return header, rows

email_file = "data/Phishing_Email.csv"
_, emails = read_csv_into_objects(email_file)
print(emails[1])

['', 'Email Text', 'Email Type']
['1', 'the other side of * galicismos * * galicismo * is a spanish term which names the improper introduction of french words which are spanish sounding and thus very deceptive to the ear . * galicismo * is often considered to be a * barbarismo * . what would be the term which designates the opposite phenomenon , that is unlawful words of spanish origin which may have crept into french ? can someone provide examples ? thank you joseph m kozono < kozonoj @ gunet . georgetown . edu >', 'Safe Email']


In [12]:
site_file = "Phishing-Dataset/dataset_full.csv"
headers, ps = read_csv_into_objects(site_file)

def load_csv_to_numpy(filename):
    data = np.genfromtxt(filename, delimiter=',', skip_header=1)  # Skip header
    return data

csv_data = load_csv_to_numpy(site_file)




['qty_dot_url', 'qty_hyphen_url', 'qty_underline_url', 'qty_slash_url', 'qty_questionmark_url', 'qty_equal_url', 'qty_at_url', 'qty_and_url', 'qty_exclamation_url', 'qty_space_url', 'qty_tilde_url', 'qty_comma_url', 'qty_plus_url', 'qty_asterisk_url', 'qty_hashtag_url', 'qty_dollar_url', 'qty_percent_url', 'qty_tld_url', 'length_url', 'qty_dot_domain', 'qty_hyphen_domain', 'qty_underline_domain', 'qty_slash_domain', 'qty_questionmark_domain', 'qty_equal_domain', 'qty_at_domain', 'qty_and_domain', 'qty_exclamation_domain', 'qty_space_domain', 'qty_tilde_domain', 'qty_comma_domain', 'qty_plus_domain', 'qty_asterisk_domain', 'qty_hashtag_domain', 'qty_dollar_domain', 'qty_percent_domain', 'qty_vowels_domain', 'domain_length', 'domain_in_ip', 'server_client_domain', 'qty_dot_directory', 'qty_hyphen_directory', 'qty_underline_directory', 'qty_slash_directory', 'qty_questionmark_directory', 'qty_equal_directory', 'qty_at_directory', 'qty_and_directory', 'qty_exclamation_directory', 'qty_spac

In [14]:
print(csv_data.shape)
file_path = 'website_data.npy'
np.save(file_path, csv_data)

(88647, 112)
